# Saraga Dataset (Three Stem Conversion)

In [1]:
import librosa as lb
import soundfile as sf
import numpy as np
import pandas as pd
import os
import shutil
from tqdm import tqdm
from pydub import AudioSegment

In [3]:
path = "/home/rajesh/Desktop/Datasets/saraga1.5_carnatic/"
top = sorted([i for i in os.listdir(path) if os.path.isdir(path+i)])
print("Top folders: ", len(top))
top_path = sorted([path+i for i in os.listdir(path) if os.path.isdir(path+i)])

Top folders:  26


In [4]:
mid_folders = []
len_sub_folders = []

mid_folders_path = []

for i in top_path:
    sub_folders = sorted([j for j in os.listdir(i) if os.path.isdir(os.path.join(i, j))])
    len_sub_folders.append(len(sub_folders))
    mid_folders.append(sub_folders)
    sub_folders_path = sorted([os.path.join(i, j) for j in os.listdir(i) if os.path.isdir(os.path.join(i, j))])
    mid_folders_path.append(sub_folders_path)

In [5]:
print('Total folders: ', sum(len_sub_folders))

Total folders:  249


In [6]:
files = []
files_path = []
for k in range(len(top)):
    for i in mid_folders_path[k]:
        file = sorted([j for j in os.listdir(i) if j[-3:] == 'mp3'])
        files.append(file)
        
        file_p = sorted([os.path.join(i, j) for j in os.listdir(i) if j[-3:] == 'mp3'])
        files_path.append(file_p)

In [7]:
len(files)

249

In [8]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
os.makedirs(dest_folder, exist_ok=True)

In [9]:
c = 1
for i in tqdm(range(len(files))):
    if len(files[i]) >= 3:
        folder_name = files[i][0].split('.')[0].replace(" ", "")
        os.makedirs(dest_folder+folder_name, exist_ok=True)
        for j in range(len(files[i])):
            name = files[i][j].split('.')
            if name[1] == 'mp3':
                shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'mixture.mp3'))
            else:
                deep_name = name[1].split('-')
                if deep_name[1] == "ghatam":
                    shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'ghatam.mp3'))
                if deep_name[1] == "violin":
                    shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'violin.mp3'))
                if deep_name[1] == "vocal":
                    try:
                        if deep_name[2] == 's':
                            shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'vocals.mp3'))
                        else:
                            shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'svocals.mp3'))
                    except:
                        shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'vocals.mp3'))
                if deep_name[1] == "mridangam":
                    if deep_name[2] == "right":
                        shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'right.mp3'))
                    else:
                        shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'left.mp3'))
        
        #print(c, "folders done")            
        c+=1
            
print(c)

100%|█████████████████████████████████████████| 249/249 [08:16<00:00,  1.99s/it]

169


In [10]:
c = 1
s_v = 0
for i in tqdm(range(len(files))):
    if len(files[i]) >= 3:
        folder_name = files[i][0].split('.')[0].replace(" ", "")
        os.makedirs(dest_folder+folder_name, exist_ok=True)
        for j in range(len(files[i])):
            name = files[i][j].split('.')
            if name[1] == 'mp3':
                pass
            else:
                deep_name = name[1].split('-')
                if deep_name[1] == "vocal":
                    if len(deep_name) == 3:
                        shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'svocals.mp3'))
                        s_v += 1
                    else:
                        shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'vocals.mp3'))
        
        #print(c, "folders done")            
        c+=1
            
print(c)
print(s_v, 'Secondary vocal copied!')

100%|█████████████████████████████████████████| 249/249 [01:32<00:00,  2.71it/s]

169
24 Secondary vocal copied!


### Converting copied .mp3 to .wav

In [12]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]
c = 1
for i in tqdm(files):
    file = os.listdir(i)
    for j in file:
        #print(i+'/'+j)
        sound = AudioSegment.from_mp3(i+'/'+j)
        sound.export(i+'/'+j[:-4]+'.wav', format="wav")
    #print('{}/161 folders done'.format(c))
    c+=1

100%|█████████████████████████████████████████| 161/161 [24:36<00:00,  9.17s/it]


In [ ]:
# Copy Vocal alone

In [13]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]
c = 1
for i in tqdm(files):
    file = os.listdir(i)
    for j in file:
        #print(i+'/'+j)
        if j[:-4] != 'mixture' and j[-3:] != 'wav':
            sound = AudioSegment.from_mp3(i+'/'+j)
            sound.export(i+'/'+j[:-4]+'.wav', format="wav")
    #print('{}/161 folders done'.format(c))
    c+=1

100%|█████████████████████████████████████████| 161/161 [18:13<00:00,  6.79s/it]


### Deleting .mp3

In [25]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]
for i in tqdm(files):
    file = os.listdir(i)
    for j in file:
        if j[-3:] == 'mp3':
            os.remove(i+'/'+j)

100%|█████████████████████████████████████████| 161/161 [00:21<00:00,  7.53it/s]


### Vocal & Vocal Secondary Mix:

In [14]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    if 'vocals.wav' in file and 'svocals.wav' in file:
        vocal1 = AudioSegment.from_file(i+'/vocals.wav', format="wav")
        vocal2 = AudioSegment.from_file(i+'/svocals.wav', format="wav")
        out = vocal1.overlay(vocal2, position=0)
        os.remove(i+'/vocals.wav')
        os.remove(i+'/svocals.wav')
        out.export(i+'/vocals.wav', format="wav")

100%|█████████████████████████████████████████| 161/161 [00:59<00:00,  2.72it/s]


### Mirdangam Left & Right Mix

In [15]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    if 'left.wav' in file and 'right.wav' in file:
        left = AudioSegment.from_file(i+'/left.wav', format="wav")
        right = AudioSegment.from_file(i+'/right.wav', format="wav")
        out = left.overlay(right, position=0)
        os.remove(i+'/left.wav')
        os.remove(i+'/right.wav')
        out.export(i+'/mridangam.wav', format="wav")

100%|█████████████████████████████████████████| 161/161 [07:02<00:00,  2.62s/it]


### Counting only ghatam

In [15]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

c = 0
for i in tqdm(files):
    file = os.listdir(i)
    if 'ghatam.wav' in file:
        c+=1
print(c, 'files available')

100%|█████████████████████████████████████████| 161/161 [00:04<00:00, 39.06it/s]

45 files available


### Counting Valid Three Stems (Vocals, Violin, Mridangam, Mixed)

In [16]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

c = 0
for i in tqdm(files):
    file = os.listdir(i)
    if 'vocals.wav' in file and 'violin.wav' in file and 'mridangam.wav' in file and 'mixture.wav' in file:
        c+=1
print(c, 'files available')

100%|██████████████████████████████████████| 161/161 [00:00<00:00, 14531.59it/s]

161 files available


### Converting Mixture from 2 channel to single channel

In [16]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

skip = 0
for i in tqdm(files):
    file = os.listdir(i)
    if 'mixture.wav' in file:
        mixture, msr = sf.read(i+'/mixture.wav')
        
        avg = []
        try:
            for j in mixture:
                avg.append((j[0] + j[1])/2)
            skip+=1 
            flag = True
        except:
            avg = mixture
            flag = False
            
        if flag == True:
            os.remove(i+'/mixture.wav')
            sf.write(i+'/mixture.wav', avg, msr)
        
print(skip, ' files skipped')

100%|███████████████████████████████████████| 161/161 [1:03:16<00:00, 23.58s/it]


161  files skipped


### Making all of same length

In [17]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    if 'vocals.wav' in file and 'violin.wav' in file and 'mridangam.wav' in file and 'mixture.wav' in file:
        vocal, vsr = sf.read(i+'/vocals.wav')
        violin, visr = sf.read(i+'/violin.wav')
        mridangam, mrsr = sf.read(i+'/mridangam.wav')
        mixture, msr = sf.read(i+'/mixture.wav')
        
        
        low = min([len(vocal), len(violin), len(mridangam), len(mixture)])
        
        mixture = mixture[:low]
        
        os.remove(i+'/mixture.wav')
        sf.write(i+'/mixture.wav', mixture, msr)

100%|█████████████████████████████████████████| 161/161 [14:22<00:00,  5.36s/it]


In [18]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    if 'vocals.wav' in file and 'violin.wav' in file and 'mridangam.wav' in file and 'mixture.wav' in file:
        vocal, vsr = sf.read(i+'/vocals.wav')
        violin, visr = sf.read(i+'/violin.wav')
        mridangam, mrsr = sf.read(i+'/mridangam.wav')
        mixture, msr = sf.read(i+'/mixture.wav')
        
        low = min([len(vocal), len(violin), len(mridangam), len(mixture)])
        
        violin = violin[:low]
        
        os.remove(i+'/violin.wav')
        sf.write(i+'/violin.wav', violin, visr)

100%|█████████████████████████████████████████| 161/161 [13:48<00:00,  5.15s/it]


In [19]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    if 'vocals.wav' in file and 'violin.wav' in file and 'mridangam.wav' in file and 'mixture.wav' in file:
        vocal, vsr = sf.read(i+'/vocals.wav')
        violin, visr = sf.read(i+'/violin.wav')
        mridangam, mrsr = sf.read(i+'/mridangam.wav')
        mixture, msr = sf.read(i+'/mixture.wav')
        
        low = min([len(vocal), len(violin), len(mridangam), len(mixture)])
        
        mridangam = mridangam[:low]
        
        os.remove(i+'/mridangam.wav')
        sf.write(i+'/mridangam.wav', mridangam, mrsr)

100%|█████████████████████████████████████████| 161/161 [14:52<00:00,  5.54s/it]


In [20]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    if 'vocals.wav' in file and 'violin.wav' in file and 'mridangam.wav' in file and 'mixture.wav' in file:
        vocal, vsr = sf.read(i+'/vocals.wav')
        violin, visr = sf.read(i+'/violin.wav')
        mridangam, mrsr = sf.read(i+'/mridangam.wav')
        mixture, msr = sf.read(i+'/mixture.wav')
        
        low = min([len(vocal), len(violin), len(mridangam), len(mixture)])
        
        vocal = vocal[:low]
        
        os.remove(i+'/vocals.wav')
        sf.write(i+'/vocals.wav', vocal, vsr)
            

100%|█████████████████████████████████████████| 161/161 [14:38<00:00,  5.46s/it]


### Deleting Ghatam

In [21]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

c = 0
for i in tqdm(files):
    file = os.listdir(i)
    if 'ghatam.wav' in file:
        c+=1
        os.remove(i+'/ghatam.wav')
print(c, 'files available')

100%|█████████████████████████████████████████| 161/161 [00:07<00:00, 22.77it/s]

45 files available


In [22]:
### Check every file is of equal length or not

In [23]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

c = 0
for i in tqdm(files):
    file = os.listdir(i)
    if 'vocals.wav' in file and 'violin.wav' in file and 'mridangam.wav' in file and 'mixture.wav' in file:
        vocal, vsr = sf.read(i+'/vocals.wav')
        violin, visr = sf.read(i+'/violin.wav')
        mridangam, mrsr = sf.read(i+'/mridangam.wav')
        mixture, msr = sf.read(i+'/mixture.wav')
        
        if len(vocal) == len(violin) == len(mridangam) == len(mixture):
            c+=1
        else:
            print(i)
            

100%|█████████████████████████████████████████| 161/161 [08:56<00:00,  3.33s/it]


In [24]:
c

161

## Errors in WaveUnet

In [7]:
c = 1
for i in tqdm(range(len(files))):
    if len(files[i]) >= 3:
        folder_name = files[i][0].split('.')[0].replace(" ", "")
        os.makedirs(dest_folder+folder_name, exist_ok=True)
        for j in range(len(files[i])):
            name = files[i][j].split('.')
            if name[1] == 'mp3':
                shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'mix.mp3'))
        c+=1
            
print(c)

100%|█████████████████████████████████████████| 249/249 [01:58<00:00,  2.10it/s]

169


In [12]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    #print(file)
    if 'mix.mp3' in file:
        sound = AudioSegment.from_mp3(i+'/mix.mp3')
        sound.export(i+'/mix.wav', format="wav")
        os.remove(i+'/mix.mp3')

100%|█████████████████████████████████████████| 161/161 [06:25<00:00,  2.39s/it]


In [13]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    #print(file)
    if 'mixture.wav' in file:
        os.remove(i+'/mixture.wav')

100%|█████████████████████████████████████████| 161/161 [00:14<00:00, 10.95it/s]


In [15]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/train/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    #print(file)
    if 'accompaniment.wav' in file:
        os.remove(i+'/accompaniment.wav')

100%|█████████████████████████████████████████| 111/111 [00:11<00:00,  9.43it/s]


In [21]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/train/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    #print(file)
    if 'mix.wav' in file:
        #print(i+'/mix.wav', i+'/mixture.wav')
        shutil.copy2(i+'/mix.wav', i+'/mixture.wav')
        #os.remove(i+'/mix.wav')

100%|█████████████████████████████████████████| 111/111 [07:09<00:00,  3.87s/it]


In [23]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/test/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    #print(file)
    if 'mix.wav' in file:
        #print(i+'/mix.wav', i+'/mixture.wav')
        #shutil.copy2(i+'/mix.wav', i+'/mixture.wav')
        os.remove(i+'/mix.wav')

100%|███████████████████████████████████████████| 50/50 [00:03<00:00, 15.90it/s]


In [26]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/train/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    if 'vocals.wav' in file and 'violin.wav' in file and 'mridangam.wav' in file and 'mixture.wav' in file:
        vocal, vsr = sf.read(i+'/vocals.wav')
        violin, visr = sf.read(i+'/violin.wav')
        mridangam, mrsr = sf.read(i+'/mridangam.wav')
        mixture, msr = sf.read(i+'/mixture.wav')
        
        low = min([len(vocal), len(violin), len(mridangam), len(mixture)])
        
        mixture = mixture[:low]
        
        #os.remove(i+'/mixture.wav')
        sf.write(i+'/mixture.wav', mixture, msr)

100%|█████████████████████████████████████████| 111/111 [12:57<00:00,  7.01s/it]
